In [ ]:
%%bash
pip install transformers
pip install torchmetrics
pip install transformers[sentencepiece]
pip install sacremoses
pip install -U deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 10.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
Looking in indexes: https://pypi.org/simple, https://us-pytho

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


In [ ]:
import torch
from transformers import BertTokenizer
from transformers import pipeline, Pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np
import pandas as pd 
from torch import nn
import torch.optim as optim
from deep_translator import GoogleTranslator


Importing all datasets!



In [ ]:
# ### just a one time thing, cleaning the datasets up: 

# kor_dataset_train = pd.read_csv("/content/MLDL_korean_dataset_train.csv").drop("Unnamed: 0", axis = 1)[["comments", "contain_gender_bias"]]
# kor_dataset_test = pd.read_csv("/content/MLDL_korean_dataset_test.csv").drop("Unnamed: 0", axis = 1)[["comments", "contain_gender_bias"]]

# kor_dataset_train = kor_dataset_train.rename(columns = {"comments": "text", "contain_gender_bias": "misogynous"})
# kor_dataset_test = kor_dataset_test.rename(columns = {"comments": "text", "contain_gender_bias": "misogynous"})

# kor_dataset_train['misogynous'] = (kor_dataset_train['misogynous']).astype(int)
# kor_dataset_test['misogynous'] = (kor_dataset_test['misogynous']).astype(int)

# kor_dataset_train.to_csv("kor_dataset_train_clean.csv")
# kor_dataset_test.to_csv("kor_dataset_test_clean.csv")

# ####################################################################################################################

# it_dataset_train = pd.read_csv("/content/it_training_anon.tsv", sep = '\t')[['text', 'misogynous']]
# it_dataset_test = pd.read_csv("/content/it_testing_labeled_anon.tsv", sep = '\t')[['text', 'misogynous']]

# words_remove = ["<MENTION_1>", "<MENTION_2>", "<MENTION_3>"]

# for i in range(len(it_dataset_train['text'])):
#     text = it_dataset_train['text'][i]
#     it_dataset_train['text'][i] = " ".join([word for word in text.split() if word not in words_remove])

# for i in range(len(it_dataset_test['text'])):
#     it_dataset_test['text'][i] = " ".join([word for word in it_dataset_test['text'][i].split() if word not in words_remove])

# it_dataset_train.to_csv("it_dataset_train_clean.csv")
# it_dataset_test.to_csv("it_dataset_test_clean.csv")

# ####################################################################################################################

# en_dataset_train = pd.read_csv("/content/en_training_anon.tsv", sep = '\t')[['text', 'misogynous']]
# en_dataset_test = pd.read_csv("/content/en_testing_labeled_anon.tsv", sep = '\t')[['text', 'misogynous']]

# words_remove = ["<MENTION_1>", "<MENTION_2>", "<MENTION_3>"]

# for i in range(len(en_dataset_train['text'])):
#     en_dataset_train['text'][i] = " ".join([word for word in en_dataset_train['text'][i].split() if word not in words_remove])

# for i in range(len(en_dataset_test['text'])):
#     en_dataset_test['text'][i] = " ".join([word for word in en_dataset_test['text'][i].split() if word not in words_remove])

# en_dataset_train.to_csv("en_dataset_train_clean.csv")
# en_dataset_test.to_csv("en_dataset_test_clean.csv")

# ####################################################################################################################

# en_dataset_alt = pd.read_csv("/content/final_labels.csv")[['body', 'level_1']]
# en_dataset_alt = en_dataset_alt.rename(columns={"level_1": "misogynous"})
# en_dataset_alt["misogynous"] = (en_dataset_alt["misogynous"] == "Misogynistic").astype(int)
# en_dataset_alt.to_csv("en_dataset_alt.csv")

Now augment the dataset. We use random insertion and back translation. We avoid using random replacement as augmentation since the word that is replaced may cause the entire meaning of the sentence to change. We use huggingface pipelines for these augmentations. 

Most importantly, we don't augment in the Dataset class since the pipeline needs to know the source language beforehand for the translation model. 


In [ ]:
def translate(df, source_language): 
    texts = list(df["text"])
    batch_size = 8

    df_texts = []
    for i in range(0, len(texts), batch_size):
        print(i)
        if np.random.random() > 0.7:
            t_text = GoogleTranslator(source=source_language, target='de').translate_batch(texts[i:i+batch_size]) 
            t_text2 = GoogleTranslator(source='de', target=source_language).translate_batch(t_text) 
            df_texts += t_text2
        else:
            df_texts += (texts[i:i+batch_size])
  
    df["text"] = df_texts
    return df

def process_text(text):
    orig_text_list = text.split()
    len_input = len(orig_text_list)
    #Random index where we want to insert the word except at the start or end
    try:
        rand_idx = np.random.randint(1,len_input-2)
      
    except: 
        rand_idx = 1

    new_text_list = orig_text_list[:rand_idx] + ['[MASK]'] + orig_text_list[rand_idx:]
    new_mask_sent = ' '.join(new_text_list)
    return new_mask_sent
     
def insert(df): 
    texts = df["text"]
    unmasker = pipeline('fill-mask', model='bert-base-multilingual-cased')
    batch_size = 8

    for i in range(0, len(texts), batch_size):
        if np.random.random() > 0.5:
            temp_texts = list(map(process_text, texts[i:i+batch_size]))

            augmented_text_list = unmasker(temp_texts)
            augmented_text = [text[0]['sequence'] for text in augmented_text_list]
            texts[i:i+batch_size] = augmented_text

    df["text"] = texts
    return df

In [ ]:
class MyPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, df, maybe_arg=2):
        df["text"] = df["text"].apply(lambda x: process_text(x))
        model_input = torch.Tensor(df["text"])
        return {"model_input": model_input}

    def _forward(self, model_inputs):
        outputs = self.model(**model_inputs)
        # Maybe {"logits": Tensor(...)}
        return outputs

    def postprocess(self, model_outputs):
        return model_outputs["model_input"]

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

class Dataset(torch.utils.data.Dataset): 
  def __init__(self, df):
    self.labels = list(df['misogynous'])
    self.texts = [tokenizer(text, padding='max_length', 
                            max_length = 512, truncation=True,
                            return_tensors="pt") for text in df['text']]

  def classes(self):
      return self.labels

  def __len__(self):
      return len(self.labels)

  def get_batch_labels(self, idx):
      # Fetch a batch of labels
      return np.array(self.labels[idx])

  def get_batch_texts(self, idx):
      # Fetch a batch of inputs
      return self.texts[idx]

  def __getitem__(self, idx):
      batch_texts = self.get_batch_texts(idx)
      batch_y = self.get_batch_labels(idx)

      return batch_texts, batch_y

In [67]:
# Unaugmented dataset

kor_dataset_train = pd.read_csv("/content/kor_dataset_train_clean.csv").drop("Unnamed: 0", axis = 1)
kor_dataset_test = pd.read_csv("/content/kor_dataset_test_clean.csv").drop("Unnamed: 0", axis = 1)
it_dataset_train = pd.read_csv("/content/it_dataset_train_clean.csv")
it_dataset_test = pd.read_csv("/content/it_dataset_test_clean.csv")
en_dataset_train = pd.read_csv("/content/en_dataset_train_clean.csv")
en_dataset_test = pd.read_csv("/content/en_dataset_test_clean.csv")

df2 = pd.concat([kor_dataset_train, it_dataset_train, en_dataset_train, kor_dataset_test, it_dataset_test, en_dataset_test]).drop("Unnamed: 0", axis = 1)
print(len(df2))
# df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])
# df_train = df_train.reset_index(drop = True)
# df_val = df_val.reset_index(drop = True)
# df_test = df_test.reset_index(drop = True)

18367


In [79]:
# Augmented dataset 

df = pd.read_csv("/content/masterdf.csv").drop("Unnamed: 0", axis=1)


df = df.loc[(df['text'].notna())]
df = pd.concat([df, df2]).drop_duplicates().reset_index(drop=True)

df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [80]:
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.ff1 = nn.Linear(768, 360)
        self.ff2 = nn.Linear(360, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigmoid(linear_output)

        return final_layer

In [81]:
from torch.optim import Adam
from tqdm import tqdm

epochs = 2
model = BertClassifier()
learning_rate = 1e-5

train, val = Dataset(df_train), Dataset(df_val)

train_dataloader = torch.utils.data.DataLoader(train, batch_size=24, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=8)

gpu_boole = torch.cuda.is_available()

criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr= learning_rate)

if gpu_boole:
    model = model.cuda()

for epoch_num in range(epochs):

    total_acc_train = 0
    total_loss_train = 0

    for train_input, train_label in tqdm(train_dataloader):
        mask = train_input['attention_mask']
        input_id = train_input['input_ids'].squeeze(1)

        if gpu_boole:
          train_label = train_label.cuda()
          mask, input_id = mask.cuda(), input_id.cuda()

        output = model(input_id, mask)
        
        batch_loss = criterion(output.squeeze(1), train_label.float())
        total_loss_train += batch_loss.item()
        
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
    
    total_acc_val = 0
    total_loss_val = 0

    with torch.no_grad():

        for val_input, val_label in val_dataloader:
            mask = val_input['attention_mask']
            input_id = val_input['input_ids'].squeeze(1)

            if gpu_boole:
                val_label = val_label.cuda()
                mask, input_id = mask.cuda(), input_id.cuda()

            output = model(input_id, mask)

            batch_loss = criterion(output.squeeze(1), val_label.float())
            total_loss_val += batch_loss.item()
            
            acc = (output.argmax(dim=1) == val_label).sum().item()
            total_acc_val += acc
    
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(df_train): .3f} \
        | Train Accuracy: {total_acc_train / len(df_train): .3f} \
        | Val Loss: {total_loss_val / len(df_val): .3f} \
        | Val Accuracy: {total_acc_val / len(df_val): .3f}')
    
    torch.cuda.empty_cache()
          

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1008/1008 [34:03<00:00,  2.03s/it]


Epochs: 1 | Train Loss:  0.015         | Train Accuracy:  0.683         | Val Loss:  0.034         | Val Accuracy:  0.692


100%|██████████| 1008/1008 [34:07<00:00,  2.03s/it]


Epochs: 2 | Train Loss:  0.008         | Train Accuracy:  0.683         | Val Loss:  0.027         | Val Accuracy:  0.692


100%|██████████| 1008/1008 [34:08<00:00,  2.03s/it]


Epochs: 3 | Train Loss:  0.003         | Train Accuracy:  0.683         | Val Loss:  0.032         | Val Accuracy:  0.692


In [82]:
from torchmetrics.classification import BinaryF1Score

test = Dataset(df_test)

test_dataloader = torch.utils.data.DataLoader(test, batch_size=8)

gpu_boole = torch.cuda.is_available()

metric = BinaryF1Score()

if gpu_boole:
    metric.cuda()
    model.cuda()

total_acc_test = 0
f1_score = 0
count = 0

with torch.no_grad():

  for test_input, test_label in test_dataloader:
      mask = test_input['attention_mask']
      input_id = test_input['input_ids'].squeeze(1)

      if gpu_boole:
        test_label = test_label.cuda()
        mask, input_id = mask.cuda(), input_id.cuda()

      output = model(input_id, mask)

      f1_score += metric(output.squeeze(1), test_label).sum().item()
      count += 1

      acc = (output.argmax(dim=1) == test_label).sum().item()
      total_acc_test += acc

print(f'Test Accuracy: {total_acc_test / len(df_test): .3f}')
print(f'F1 Score: {f1_score / count: .3f}')

Test Accuracy:  0.674
F1 Score:  0.817


In [66]:
torch.cuda.empty_cache()